In [75]:
#!pip install -qU langchain_experimental langchain_openai langchain_community langchain ragas chromadb langchain-groq fastembed pypdf openai konlpy sentence-transformers

In [76]:
#!pip install python-docx



In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, SpacyTextSplitter, NLTKTextSplitter, split_text_on_tokens, SentenceTransformersTokenTextSplitter, LatexTextSplitter
from langchain.text_splitter import PythonCodeTextSplitter, KonlpyTextSplitter, ElementType

from langchain.document_loaders import PyPDFLoader
import torch
import re
from docx import Document as worddoc

from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_core.documents.base import Document

In [44]:


class TextChunker:
    def __init__(self):
        pass

    def extract_text_from_pdf(self, path = "/"):
        text = ""
        loader = PyPDFLoader(path)
        documents = loader.load()
        for doc in documents:
            text = text + "\n" + str(doc.page_content)
        return text,documents

    def char_count_chunking_with_overlap(self, text, chunk_size=200, chunk_overlap=50, splitter_type = "CharacterTextSplitter"):
        # Instantiate the CharacterTextSplitter class

        text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter class
           text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        elif isinstance(text, list):
          try:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]
          except:
            chunks = text_splitter.split_documents(text)
        return chunks

    #Nonoverlap
    def char_count_chunking_with_nonoveralp(self, text, chunk_size=200, splitter_type = "CharacterTextSplitter"):
       # Instantiate the CharacterTextSplitter class
        text_splitter = CharacterTextSplitter(chunk_size=chunk_size)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter clas
           text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(chunk_size=chunk_size)
        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        elif isinstance(text, list):
          try:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]
          except:
            chunks = text_splitter.split_documents(text)

        return chunks

    #chunking
    def char_count_chunking_with_custom_delimiter(self, text, chunk_size=200, chunk_overlap=50, delimiter="\n\n",splitter_type = "CharacterTextSplitter"):
        # Instantiate the CharacterTextSplitter class
        text_splitter = CharacterTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        if splitter_type ==  "RecursiveCharacterTextSplitter":# Instantiate the RecursiveCharacterTextSplitter class
           text_splitter = RecursiveCharacterTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "NLTKTextSplitter":
           text_splitter = NLTKTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SpacyTextSplitter":
           text_splitter = SpacyTextSplitter(separator=delimiter, chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "SentenceTransformersTokenTextSplitter":
           text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "LatexTextSplitter":
           text_splitter = LatexTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "PythonCodeTextSplitter":
           text_splitter = PythonCodeTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        elif splitter_type == "KonlpyTextSplitter":
           text_splitter = KonlpyTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

        # Create documents using the text splitter
        docs =[]
        chunks = []
        if isinstance(text, str):
          docs = text_splitter.create_documents([text])
          chunks = [doc.page_content for doc in docs]
        else:
          try:
            chunks = text_splitter.split_documents(text)
          except:
            docs = text_splitter.create_documents(text)
            chunks = [doc.page_content for doc in docs]

        return chunks

    #Semantic chunking
    def semantic_section_chunking(self, documents , text_embedding_model_name,  breakpoint_threshold_type = "percentile"):
        embed_model = FastEmbedEmbeddings(model_name = text_embedding_model_name)
        semantic_chunker = SemanticChunker(embed_model, breakpoint_threshold_type=breakpoint_threshold_type)
        try:
          semantic_chunks = semantic_chunker.create_documents([d.page_content for d in documents])
        except:
          semantic_chunks = semantic_chunker.create_documents([d for d in markdown_splits])
        return semantic_chunks

    #Markdown HeaderTextSplitter
    def markdown_header_textsplitter(self, text, headers_to_split_on, strip_headers):
        from langchain.text_splitter import MarkdownHeaderTextSplitter

        markdown_document = text
        markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,  strip_headers=strip_headers)
        md_header_splits = markdown_splitter.split_text(markdown_document)
        return md_header_splits

    #Markdown TextSplitter
    def markdown_textsplitter(self, text):
        from langchain.text_splitter import MarkdownTextSplitter
        splitter = MarkdownTextSplitter()
        split_text = splitter.split_text(text)
        return split_text

    #ouputs to Doc
    #from docx import Document
    #import re
    def save_as_word_doc(self, split_text, file_path ,heading):
        doc = worddoc()
        doc.add_heading(heading, level=1)
        for i, text_chunk in enumerate(split_text):
            text_chunk= re.sub(r'[\x00-\x1F]+', '', str(text_chunk))
            doc.add_heading(f'Chunk {i + 1}', level=1)
            doc.add_paragraph(text_chunk)
        # Save the document
        doc.save(file_path)
        print("Output saved as doc successfully")



#Usage
pdf_path =r"/content/drive/MyDrive/chunking/ds (1).pdf"

text_chunker = TextChunker()
text, documents  = text_chunker.extract_text_from_pdf(path = pdf_path )

In [26]:
#MarkdownTextSplitter

split_text = text_chunker.markdown_textsplitter(text)
file_name = "MarkdownTextSplitter_chunks.docx"
file_path = "/content/drive/MyDrive/chunking/"
heading = "MarkdownTextSplitter_chunks"

text_chunker.save_as_word_doc(split_text, file_path + file_name ,heading)
# for i, text_chunk in enumerate(split_text):
#     print("########################################################################")
#     print(f"Chunk {i + 1}:\n{text_chunk}\n")



Output saved as doc successfully


In [30]:
#SentenceTransformersTokenTextSplitter chunking
split_text = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")

file_name = "SentenceTransformersTokenTextSplitter.docx"
file_path = "/content/drive/MyDrive/chunking/"
heading = "SentenceTransformersTokenTextSplitter"

text_chunker.save_as_word_doc(split_text, file_path + file_name ,heading)
# for i, text_chunk in enumerate(split_text):
#     print("########################################################################")
#     print(f"Chunk {i + 1}:\n{text_chunk}\n")


Output saved as doc successfully


In [32]:
#MarkdownTextSplitter with SentenceTransformersTokenTextSplitter chunking

file_name = "MarkdownTextSplitter_with_SentenceTransformersTokenTextSplitter_chunking.docx"
file_path = "/content/drive/MyDrive/chunking/"
heading = "MarkdownTextSplitter with SentenceTransformersTokenTextSplitter chunking"

markdown_split_text = text_chunker.markdown_textsplitter(text)
text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=200, chunk_overlap=30)

SentenceTransformers_split_text = text_chunker.char_count_chunking_with_custom_delimiter(markdown_split_text, chunk_size=200, chunk_overlap=50, splitter_type = "SentenceTransformersTokenTextSplitter")

text_chunker.save_as_word_doc(SentenceTransformers_split_text, file_path + file_name ,heading)

# for i, text_chunk in enumerate(SentenceTransformers_split_text):
#     print("########################################################################")
#     print(f"Chunk {i + 1}:\n{text_chunk}\n")

Output saved as doc successfully


In [33]:
#MarkdownTextSplitter with CharacterTextSplitter chunking

file_name = "MarkdownTextSplitter_with_CharacterTextSplitter_chunking.docx"
file_path = "/content/drive/MyDrive/chunking/"
heading = "MarkdownTextSplitter with CharacterTextSplitter chunking"

markdown_split_text = text_chunker.markdown_textsplitter(text)

split_text = text_chunker.char_count_chunking_with_overlap( markdown_split_text, chunk_size=200, chunk_overlap=50, splitter_type = "CharacterTextSplitter")


text_chunker.save_as_word_doc(split_text, file_path + file_name ,heading)

# for i, text_chunk in enumerate(SentenceTransformers_split_text):
#     print("########################################################################")
#     print(f"Chunk {i + 1}:\n{text_chunk}\n")

Output saved as doc successfully


In [34]:
#MarkdownTextSplitter with Semantic chunking

file_name = "MarkdownTextSplitter_with_Semantic_chunking.docx"
file_path = "/content/drive/MyDrive/chunking/"
heading = "MarkdownTextSplitter with Semantic chunking"

markdown_splits = text_chunker.markdown_textsplitter(text)

text_embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

semantic_chunks = text_chunker.semantic_section_chunking( markdown_splits , text_embedding_model_name,  breakpoint_threshold_type = "percentile")


text_chunker.save_as_word_doc(semantic_chunks, file_path + file_name ,heading)

# for i, text_chunk in enumerate(semantic_chunks ):
#     print("########################################################################")
#     print(f"Chunk {i + 1}:\n{text_chunk}\n")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Output saved as doc successfully


In [36]:
#MarkdownHeaderTextSplitter with CharacterTextSplitter

headers_to_split_on = [
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
            ("####", "Header 4"),
        ]
strip_headers = False

md_header_splits = text_chunker.markdown_header_textsplitter(text, headers_to_split_on, strip_headers)


md_header_splits

#You can use any other chuknikng startegies after this  MarkdownHeaderTextSplitter
chunks = text_chunker.char_count_chunking_with_nonoveralp(md_header_splits , chunk_size=200, splitter_type = "CharacterTextSplitter")


print("chunking _with_ _with_custom_delimiter with MarkdownHeaderTextSplitter:")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk}")

chunking _with_ _with_custom_delimiter with MarkdownHeaderTextSplitter:
Chunk 1: page_content='Mathematical Foundations of Data Sciences\nGabriel Peyr´ e\nCNRS & DMA\n´Ecole Normale Sup´ erieure\ngabriel.peyre@ens.fr\nhttps://mathematical-tours.github.io\nwww.numerical-tours.com\nAugust 14, 2019\n2\nChapter 1\nOptimal Transport\n1.1 Radon Measures\nMeasures. We will interchangeably the term histogram or probability vector for any element a∈Σnthat\nbelongs to the probability simplex\nΣndef.={\na∈Rn\n+;n∑\ni=1ai= 1}\n.\nA discrete measure with weights aand locations x1,...,xn∈X reads\nα=n∑\ni=1aiδxi (1.1)\nwhereδxis the Dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location\nx. Such as measure describes a probability measure if, additionally, a∈Σn, and more generally a positive\nmeasure if each of the “weights” described in vector ais positive itself.\nRemark 1 (General measures) .A convenient feature of OT is that it can deal with discrete and contin

In [ ]:
#semantic chunking MarkdownHeaderTextSplitter
'''
The effectiveness of semantic chunking techniques depends on various factors such as the nature of your data, your specific use case, and the desired outcomes.
The approach we're using, which combines Markdown header-based splitting with text embeddings for semantic analysis, is a reasonable method, especially if we're
dealing with Markdown documents or structured text like README files or articles where people use headers to organize their content. while semantic analysis is powerful for understanding text meaning, it's often most effective when combined with other techniques like the one you're using, which leverages document structure (e.g., headers) to provide additional context. By combining both approaches, you can achieve a more robust understanding of the text and its structure.
'''

headers_to_split_on = [
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
            ("####", "Header 4"),
        ]
strip_headers = False

md_header_splits = text_chunker.markdown_header_textsplitter(text, headers_to_split_on, strip_headers)

cdocs = []
for doc in md_header_splits:
    document = Document(page_content=doc.page_content)
    cdocs.append(document)


text_embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"


semantic_chunks = text_chunker.semantic_section_chunking( md_header_splits , text_embedding_model_name,  breakpoint_threshold_type = "percentile")


print("\nsemantic chunking with text embeddings:")
for i, chunk in enumerate(semantic_chunks):
    print(f"Chunk {i+1}: {chunk}")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
#MarkdownTextSplitter

split_text = text_chunker.markdown_textsplitter(text)
for i, text_chunk in enumerate(split_text):
    print(f"Chunk {i + 1}:\n{text_chunk}\n")


Chunk 1:
Mathematical Foundations of Data Sciences
Gabriel Peyr´ e
CNRS & DMA
´Ecole Normale Sup´ erieure
gabriel.peyre@ens.fr
https://mathematical-tours.github.io
www.numerical-tours.com
August 14, 2019
2
Chapter 1
Optimal Transport
1.1 Radon Measures
Measures. We will interchangeably the term histogram or probability vector for any element a∈Σnthat
belongs to the probability simplex
Σndef.={
a∈Rn
+;n∑
i=1ai= 1}
.
A discrete measure with weights aand locations x1,...,xn∈X reads
α=n∑
i=1aiδxi (1.1)
whereδxis the Dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location
x. Such as measure describes a probability measure if, additionally, a∈Σn, and more generally a positive
measure if each of the “weights” described in vector ais positive itself.
Remark 1 (General measures) .A convenient feature of OT is that it can deal with discrete and continuous
“objects” within the same framework. Such objects only need to be modelled as measures. This corresponds
t

In [ ]:


#chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50,splitter_type = "PythonCodeTextSplitter")
chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50, splitter_type = "KonlpyTextSplitter")
print("Char count chunking _with_ _with_custom_delimiter:")
for i, chunk in enumerate(chunks_char_count):
    print(f"Chunk {i+1}: {chunk}")

Char count chunking _with_ _with_custom_delimiter:
Chunk 1: Mathematical Foundations of Data Sciences Gabriel Peyr´ e CNRS & DMA ´Ecole Normale Sup´ erieure gabriel.peyre @ens .fr https: //mathematical-tours .github .io www.numerical-tours .com August 14, 2019 2 Chapter 1 Optimal Transport 1.1 Radon Measures Measures. We will interchangeably the term histogram or probability vector for any element a∈ Σnthat belongs to the probability simplex Σndef.={ a∈Rn + ;n∑ i=1ai= 1} . A discrete measure with weights aand locations x1,... ,xn ∈X reads α =n∑ i=1ai δxi (1.1) whereδxis the Dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location x. Such as measure describes a probability measure if, additionally, a∈ Σn, and more generally a positive measure if each of the “weights” described in vector ais positive itself. Remark 1 (General measures) .A convenient feature of OT is that it can deal with discrete and continuous “objects” within the same framework. Such 

In [ ]:


#chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50,splitter_type = "PythonCodeTextSplitter")
chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50, splitter_type = "PythonCodeTextSplitter")
print("Char count chunking _with_ _with_custom_delimiter:")
for i, chunk in enumerate(chunks_char_count):
    print(f"Chunk {i+1}: {chunk}")

Char count chunking _with_ _with_custom_delimiter:
Chunk 1: Mathematical Foundations of Data Sciences
Gabriel Peyr´ e
CNRS & DMA
´Ecole Normale Sup´ erieure
gabriel.peyre@ens.fr
https://mathematical-tours.github.io
www.numerical-tours.com
August 14, 2019
2
Chunk 2: www.numerical-tours.com
August 14, 2019
2
Chapter 1
Optimal Transport
1.1 Radon Measures
Measures. We will interchangeably the term histogram or probability vector for any element a∈Σnthat
Chunk 3: belongs to the probability simplex
Σndef.={
a∈Rn
+;n∑
i=1ai= 1}
.
A discrete measure with weights aand locations x1,...,xn∈X reads
α=n∑
i=1aiδxi (1.1)
Chunk 4: α=n∑
i=1aiδxi (1.1)
whereδxis the Dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location
Chunk 5: x. Such as measure describes a probability measure if, additionally, a∈Σn, and more generally a positive
measure if each of the “weights” described in vector ais positive itself.
Chunk 6: Remark 1 (General measures) .A convenient feature of 

In [ ]:
#char_count_chunking_with_custom_delimiter with semantic chunking


chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50, splitter_type = "PythonCodeTextSplitter") #You can use any splitter
# Create a Document instance
cdocs = []
for doc in chunks_char_count:
    document = Document(page_content=doc)
    cdocs.append(document)


text_embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"


semantic_chunks = text_chunker.semantic_section_chunking( cdocs , text_embedding_model_name,  breakpoint_threshold_type = "percentile")


print("\nsemantic chunking with text embeddings:")
for i, chunk in enumerate(semantic_chunks):
    print(f"Chunk {i+1}: {chunk}")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]


semantic chunking with text embeddings:
Chunk 1: page_content='Mathematical Foundations of Data Sciences\nGabriel Peyr´ e\nCNRS & DMA\n´Ecole Normale Sup´ erieure\ngabriel.peyre@ens.fr\nhttps://mathematical-tours.github.io\nwww.numerical-tours.com\nAugust 14, 2019\n2'
Chunk 2: page_content='www.numerical-tours.com\nAugust 14, 2019\n2\nChapter 1\nOptimal Transport\n1.1 Radon Measures\nMeasures. We will interchangeably the term histogram or probability vector for any element a∈Σnthat'
Chunk 3: page_content='belongs to the probability simplex\nΣndef.={\na∈Rn\n+;n∑\ni=1ai= 1}\n. A discrete measure with weights aand locations x1,...,xn∈X reads\nα=n∑\ni=1aiδxi (1.1)'
Chunk 4: page_content='α=n∑\ni=1aiδxi (1.1)\nwhereδxis the Dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location'
Chunk 5: page_content='x. Such as measure describes a probability measure if, additionally, a∈Σn, and more generally a positive\nmeasure if each of the “weights” described in ve

In [ ]:
chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50,splitter_type = "LatexTextSplitter")
print("Char count chunking _with_ _with_custom_delimiter:")
for i, chunk in enumerate(chunks_char_count):
    print(f"Chunk {i+1}: {chunk}")

Char count chunking _with_ _with_custom_delimiter:
Chunk 1: Mathematical Foundations of Data Sciences
Gabriel Peyr´ e
CNRS & DMA
´Ecole Normale Sup´ erieure
gabriel.peyre@ens.fr
https://mathematical-tours.github.io
www.numerical-tours.com
August 14,
Chunk 2: 14, 2019
2
Chapter 1
Optimal Transport
1.1 Radon Measures
Measures. We will interchangeably the term histogram or probability vector for any element a∈Σnthat
belongs to the probability
Chunk 3: any element a∈Σnthat
belongs to the probability simplex
Σndef.={
a∈Rn
+;n∑
i=1ai= 1}
.
A discrete measure with weights aand locations x1,...,xn∈X reads
α=n∑
i=1aiδxi (1.1)
whereδxis the Dirac at
Chunk 4: reads
α=n∑
i=1aiδxi (1.1)
whereδxis the Dirac at position x, intuitively a unit of mass which is inﬁnitely concentrated at location
x. Such as measure describes a probability measure if,
Chunk 5: as measure describes a probability measure if, additionally, a∈Σn, and more generally a positive
measure if each of the “weights” described in vect

In [ ]:

chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50,splitter_type = "SentenceTransformersTokenTextSplitter")
print("Char count chunking _with_ _with_custom_delimiter:")
for i, chunk in enumerate(chunks_char_count):
    print(f"Chunk {i+1}: {chunk}")


In [ ]:
chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50, delimiter="\n\n",splitter_type = "SpacyTextSplitter")
print("Char count chunking _with_ _with_custom_delimiter:")
for i, chunk in enumerate(chunks_char_count):
    print(f"Chunk {i+1}: {chunk}")

In [ ]:
chunks_char_count = text_chunker.char_count_chunking_with_overlap(text, chunk_size=500, chunk_overlap=40,splitter_type = "SpacyTextSplitter")
print("Char count chunking _with_overlap:")
for i, chunk in enumerate(chunks_char_count):
    print(f"Chunk {i+1}: {chunk}")

In [ ]:
chunks_char_count = text_chunker.char_count_chunking_with_nonoveralp(text, chunk_size=500,splitter_type = "SpacyTextSplitter")
print("Char count chunking _with_nonoverlap:")
for i, chunk in enumerate(chunks_char_count):
    print(f"Chunk {i+1}: {chunk}")

In [ ]:
chunks_char_count = text_chunker.char_count_chunking_with_custom_delimiter( text, chunk_size=200, chunk_overlap=50, delimiter="\n\n",splitter_type = "SpacyTextSplitter")
print("Char count chunking _with_ _with_custom_delimiter:")
for i, chunk in enumerate(chunks_char_count):
    print(f"Chunk {i+1}: {chunk}")

In [ ]:
text_embedding_model_name = "BAAI/bge-base-en-v1.5"

semantic_chunks = text_chunker.semantic_section_chunking( documents , text_embedding_model_name,  breakpoint_threshold_type = "percentile")


print("\nsemantic chunking with text embeddings:")
for i, chunk in enumerate(semantic_chunks):
    print(f"Chunk {i+1}: {chunk}")

In [ ]:
text_embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"

semantic_chunks = text_chunker.semantic_section_chunking( documents , text_embedding_model_name,  breakpoint_threshold_type = "percentile")


print("\nsemantic chunking with text embeddings:")
for i, chunk in enumerate(semantic_chunks):
    print(f"Chunk {i+1}: {chunk}")

In [ ]:

#HTMLHeaderTextSplitter

from langchain_text_splitters.html import HTMLHeaderTextSplitter

# Sample HTML document
html_document = """
<html>
<head>
    <title>Sample HTML Document</title>
</head>
<body>
    <h1>Introduction</h1>
    <p>This is the introductory paragraph.</p>

    <h2>Section 1</h2>
    <p>This is the content of section 1.</p>

    <h2>Section 2</h2>
    <p>This is the content of section 2.</p>

    <h3>Subsection 2.1</h3>
    <p>This is the content of subsection 2.1.</p>

    <h2>Section 3</h2>
    <p>This is the content of section 3.</p>
</body>
</html>
"""

# Initialize the HTMLHeaderTextSplitter with header tags to split on
splitter = HTMLHeaderTextSplitter(headers_to_split_on=["h1", "h2", "h3"])

# Split the HTML document
split_chunks = splitter.split_text(html_document)

# Print the split chunks
for i, chunk in enumerate(split_chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n")


Chunk 1:
page_content='This is the introductory paragraph.  \nThis is the content of section 1.  \nThis is the content of section 2.  \nThis is the content of subsection 2.1.  \nThis is the content of section 3.'

